# 양자화에 필요한 패키지 설치

In [3]:
from tqdm import trange, notebook

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import os
from peft import LoraConfig, PeftModel

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [13]:
model = AutoModelForCausalLM.from_pretrained(base_model_name, quantization_config=bnb_config, device_map={'': 0}, use_auth_token=os.environ["huggingface_token"])
model.config.use_cache = False
model.config.pretraining_tp = 1

hf_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True, use_auth_token=os.environ["huggingface_token"])
hf_tokenizer.pad_token = hf_tokenizer.eos_token
hf_tokenizer.padding_side = "right"

model = PeftModel.from_pretrained(model, f'/home/ksy/test/fine2/Easy-Fine-tuner/lora/beomi/Yi-Ko-6B_test')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


ValueError: Can't find 'adapter_config.json' at '/home/ksy/test/fine2/Easy-Fine-tuner/lora/beomi/Yi-Ko-6B_test'

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.schema.runnable import RunnablePassthrough

In [15]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [16]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import pipeline
from langchain.chains import LLMChain

text_generation_pipeline = pipeline(task="text-generation",
                model=model,
                tokenizer=hf_tokenizer,
                max_new_tokens=200,
                do_sample=True,
                temperature=0.1,
                num_return_sequences=1,
                top_k=3,
                top_p=0.3,
                repetition_penalty = 1.1,
                eos_token_id=tokenizer.eos_token_id,
                framework='pt'
)
combine_template = "Write a summary of the following text:\n\n{summaries}"
combine_prompt_template = PromptTemplate.from_template(template=combine_template)
question_template = 
"""
질문에 대한 답변을 단 하나의 문장으로 제공해주세요. 
모든 세부사항과 설명은 생략해야 합니다.
예: '메이머스트의 직원수는 약 90명입니다.'

여기 도움이 될 만한 배경지식이 있습니다:

{context}

질문: {question}
답변 형식: [정보]
"""


question_prompt_template = PromptTemplate.from_template(template=question_template)

chain_type_kwargs= {"question_prompt": question_prompt_template, "combine_prompt": combine_prompt_template}
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [17]:
import tiktoken
tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [18]:
loader = PyPDFLoader("/home/ksy/test/myenv/aa.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50,length_function = tiktoken_len)
texts = text_splitter.split_documents(pages)

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

db = FAISS.from_documents(texts, hf)

retriever = db.as_retriever(
                            search_type="similarity",
                            search_kwargs={'k': 1}
                        )

In [19]:
from langchain.chains import RetrievalQAWithSourcesChain

RAG_Chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm,
    chain_type="map_reduce",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
    return_source_documents=True,
    reduce_k_below_max_tokens=True
)


In [20]:
result = RAG_Chain("㈜메이머스트 중식보조비는 얼마인가요?")

print(f"질문\n{result['question']}\n\n답변{result['answer']}\n\n출처\n{result['source_documents']}")

/home/ksy/test/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


질문
㈜메이머스트 중식보조비는 얼마인가요?

답변

The following is a summary of the above text:

중식 보조비는 5,000원/일입니다.

출처
[Document(page_content='9\nⅢ. 복리후생규정_ 중식보조비, 급식비\n1. 중식 보조비 : 5,000원/일\n√. 임•직원에게 양질의 중식을 제공하기 위한 보조 비용\n√. 2024년부터 중식보조비를 하루 3,000원에서 5,000원으로 상향 조정\n√. 근무일 기준으로 중식 보조비를 지급하며, 매월( 25일) 급여계좌로 이체한다.\n√. 휴가(반차), 휴직, 공가, 교육(식사 제공 시) 등 비근무일에는 지급하지 않는다.\n2. 급식비 : \\15,000원/1식\n√. 업무시간 외에 회사업무로 인한 (야간/휴일)근무 시 급식을 제공하기 위한 비용\n√. 15,000원/1식 기준 한도이며 초과시 개인이 부담한다.\n√. 법인카드 결제를 원칙으로 하며 다수가 같이 식사하는 경우에도 각자 본인의 법인카드로 결제 .\n√. 급식비(야근 식대)의 개인카드 결제는 허용하지 않는다.', metadata={'source': '/home/ksy/test/myenv/aa.pdf', 'page': 8})]


In [125]:
import time
def rag(answer):
    start = time.time()
    result = RAG_Chain(answer)
    end = time.time()
    token_time = end-start
    
    print(f"답변: {result['answer']}")
    print(f"토큰생성시간: {token_time:.2f}")
    encoded_input = hf_tokenizer(result['answer'], return_tensors='pt')
    token_count = len(encoded_input['input_ids'][0])
    print(f"토큰 수: {token_count}")
    print(f"초당 토큰 수:{token_count/token_time:.2f}")

In [126]:
rag("㈜메이머스트 중식보조비는 얼마인가요?")

답변: 

The following is a summary of the above text:

중식 보조비는 5,000원/일입니다.
토큰생성시간: 3.76
토큰 수: 31
초당 토큰 수:8.25


In [21]:
def gradio_rag(answer):
    result = RAG_Chain(answer)
    return result['answer']

In [22]:
import gradio

gradio.Interface( gradio_rag, 'text', 'text',).launch(share=True)


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://47c2e2289ab495f0c2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
